In [8]:
%cd course-eval/Bart-Large-CNN
# !pwd

[Errno 2] No such file or directory: 'course-eval/Bart-Large-CNN'
/home/thanapon.nor/course-eval/Bart-Large-CNN


In [14]:
ls

__pycache__/        runs/                  train.ipynb
bart-large-cnn.zip  seq2seq_model_M.ipynb  train_MAMS_sentiment.ipynb
cache_dir/          seq2seq_model_M.py     train_MAMS_sentiment.py
cp-remover.ipynb    test_MAMS.ipynb
outputs/            test_MAMS.py


In [5]:
from simpletransformers.seq2seq import Seq2SeqModel
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig, GPT2Tokenizer, GPT2Model, MBart50TokenizerFast, MBartForConditionalGeneration, GPT2LMHeadModel
# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)
import torch
import torch.nn as nn
import numpy as np

In [9]:
fold = "1"

# Original, Back-Translation
model_type = "Original"

path = f"../Dataset/{model_type}/Splited/{fold}-fold/testformat.txt"
model_path = f"/outputs/{model_type}/{fold}-fold"

print(f"Test Path: {path}\nModel Path: {model_path}")

Test Path: ../Dataset/Original/Splited/1-fold/testformat.txt
Model Path: /outputs/Original/1-fold


In [11]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# if torch.cuda.is_available():
#     print(f"CUDA devices available: {torch.cuda.device_count()}")
#     print(f"Current CUDA device index: {torch.cuda.current_device()}")
#     print(f"Current CUDA device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
# else:
#     print("CUDA is not available. Switch to CPU mode or check your CUDA installation.")

# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# device

CUDA devices available: 1
Current CUDA device index: 0
Current CUDA device name: NVIDIA RTX A6000


device(type='cuda', index=1)

In [12]:

#def calculate_metrics(predicted, golden):
#    label_mapping = {"positive": 1, "neutral": 2, "negative": 0}

    # Map labels to numeric values and handle 'nan' values
#    golden_numeric = np.array([label_mapping.get(label, -1) for label in golden])

    # Filter out entries with -1 (nan) before further calculations
#    valid_entries = golden_numeric != -1
#    predicted_numeric = np.array([label_mapping[label] for label in predicted])[valid_entries]
#    golden_numeric = golden_numeric[valid_entries]

#    TP = np.sum(np.equal(predicted_numeric, golden_numeric) & np.equal(predicted_numeric, 1))
#    FP = np.sum(np.not_equal(predicted_numeric, golden_numeric) & np.equal(predicted_numeric, 1))
#    FN = np.sum(np.not_equal(predicted_numeric, golden_numeric) & np.equal(predicted_numeric, 0))
#    TN = np.sum(np.equal(predicted_numeric, golden_numeric) & np.equal(predicted_numeric, 0))

#    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
#    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
#    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

#    return precision, recall, f1_score


In [10]:
def predict_val(model, device):#, output_dir):
    candidate_list = [ "positive", "neutral", "negative"]

    #model = MBartForConditionalGeneration.from_pretrained(output_dir)
    #model = MBartForConditionalGeneration.from_pretrained('/ICTeval/ICTeval_code/outputs/mbart')
    model.eval()
    model.config.use_cache = False
    #tokenizer = MBart50TokenizerFast.from_pretrained('facebook/mbart-large-50')
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    #tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/mGPT")
    with open(path, "r") as f:
        file = f.readlines()
    train_data = []
    count = 0
    total = 0
    true_labels = []
    predicted_labels = []

   #with torch.no_grad():
   #     for module in model.modules():
   #         if isinstance(module, nn.Dropout):
   #             module.training = False
   #         elif isinstance(module, nn.BatchNorm1d):
   #             module.eval()

    for line in file:
        total += 1
        # score_list = []
        score_list1 = []
        score_list2 = []
        score_list3 = []
        score_list4 = []
        score_list5 = []
        line = line.strip()
        x, term, golden_polarity = line.split("\001")[0], line.split("\001")[1], line.split("\001")[2]
        input_ids = tokenizer([x] * 3, return_tensors='pt')['input_ids'].to(device)

        # target_list = ["For " + term.lower() + ", the sentiment is " + candi.lower() + " ." for candi in candidate_list]
        # # target_list = ["For " + term.lower() + ", it is a " + candi.lower() + "sentence ." for candi in candidate_list]
        # # input_ids = tokenizer([x] * 3, return_tensors='pt')['input_ids']
        # output_ids = tokenizer(target_list, return_tensors='pt', padding=True, truncation=True)['input_ids']
        # with torch.no_grad():
        #     output = model(input_ids=input_ids.to(device), decoder_input_ids=output_ids.to(device))[0]
        #     logits = output.softmax(dim=-1).to('cpu').numpy()
        # for i in range(3):
        #     score = 1
        #     for j in range(logits[i].shape[0] - 2):
        #         score *= logits[i][j][output_ids[i][j + 1]]
        #     score_list1.append(score)

        target_list = ["The sentiment polarity of " + term.lower() + " is " + candi.lower() + " ." for candi in
                       candidate_list]
        ## target_list = ["For " + term.lower() + ", it is a " + candi.lower() + "sentence ." for candi in candidate_list]
        ## input_ids = tokenizer([x] * 3, return_tensors='pt')['input_ids']

        output_ids = tokenizer(target_list, return_tensors='pt', padding=True, truncation=True)['input_ids'].to(device)
        model.to(device)
        output_ids = output_ids.to(device)
        with torch.no_grad():
            output = model(input_ids=input_ids, decoder_input_ids=output_ids)[0]
            logits = output.softmax(dim=-1).to('cpu').numpy()
        list_score = []
        #with torch.no_grad():
        #    output = model(input_ids=input_ids.to(device))[0]
        #    logits = output.softmax(dim=-1).to('cpu').numpy()
        for i in range(3):
            score = 0
            for j in range(logits[i].shape[0] - 2):
                #print(logits[i][j][output_ids[i][j + 1]])
                score += np.log(logits[i][j][output_ids[i][j + 1]])
                list_score.append(np.log(logits[i][j][output_ids[i][j + 1]]))
            score_list2.append(score)
        score_list = score_list2
        #print("log lsit:",np.array(list_score).reshape(3,-1))

        #loss = -torch.log(torch.tensor(score_list)).mean().item()  # Calculate loss for the example
        #losses.append(loss)

        # target_list = ["The " + term.lower() + " category has a " + candi.lower() + " label ." for candi in
        #                candidate_list]
        # # target_list = ["For " + term.lower() + ", it is a " + candi.lower() + "sentence ." for candi in candidate_list]
        # # input_ids = tokenizer([x] * 3, return_tensors='pt')['input_ids']
        # output_ids = tokenizer(target_list, return_tensors='pt', padding=True, truncation=True)['input_ids']
        # with torch.no_grad():
        #     output = model(input_ids=input_ids.to(device), decoder_input_ids=output_ids.to(device))[0]
        #     logits = output.softmax(dim=-1).to('cpu').numpy()
        # for i in range(3):
        #     score = 1
        #     for j in range(logits[i].shape[0] - 2):
        #         score *= logits[i][j][output_ids[i][j + 1]]
        #     score_list3.append(score)

        # target_list = ["The sentiment is " + candi.lower() + " for " + term.lower() + " ." for candi in
        #                candidate_list]
        # # target_list = ["For " + term.lower() + ", it is a " + candi.lower() + "sentence ." for candi in candidate_list]
        # # input_ids = tokenizer([x] * 3, return_tensors='pt')['input_ids']
        # output_ids = tokenizer(target_list, return_tensors='pt', padding=True, truncation=True)['input_ids']
        # with torch.no_grad():
        #     output = model(input_ids=input_ids.to(device), decoder_input_ids=output_ids.to(device))[0]
        #     logits = output.softmax(dim=-1).to('cpu').numpy()
        # for i in range(3):
        #     score = 1
        #     for j in range(logits[i].shape[0] - 2):
        #         score *= logits[i][j][output_ids[i][j + 1]]
        #     score_list4.append(score)

        # target_list = ["The " + term.lower() + " is " + candi.lower() + " ." for candi in
        #                candidate_list]
        # # target_list = ["For " + term.lower() + ", it is a " + candi.lower() + "sentence ." for candi in candidate_list]
        # # input_ids = tokenizer([x] * 3, return_tensors='pt')['input_ids']
        # output_ids = tokenizer(target_list, return_tensors='pt', padding=True, truncation=True)['input_ids']
        # with torch.no_grad():
        #     output = model(input_ids=input_ids.to(device), decoder_input_ids=output_ids.to(device))[0]
        #     logits = output.softmax(dim=-1).to('cpu').numpy()
        # for i in range(3):
        #     score = 1
        #     for j in range(logits[i].shape[0] - 2):
        #         score *= logits[i][j][output_ids[i][j + 1]]
        #     score_list5.append(score)

        # score_list = [(score_list1[i] + score_list2[i] + score_list3[i]) for i in range(0, len(score_list1))]
        print(score_list)
        predict = candidate_list[np.argmax(score_list)]
        predicted_sentence = target_list[np.argmax(score_list)]
        true_labels.append(golden_polarity)
        predicted_labels.append(predict)
        #predicted_term = term.lower()
        if predict == golden_polarity:
            count += 1
            print(line, predicted_sentence, " actual:", golden_polarity, "acc:", count/total, count, total)
        else:
            print("Mismatch:",line," ", predicted_sentence, " actual:", golden_polarity,"acc:", count/total, count, total)

    #matrix = self.compute_metrics(np.array(true_labels), np.array(predicted_labels), calculate_metrics=calculate_metrics)
    #precision, recall, f1_score = matrix['calculate_metrics']
    #precision, recall, f1_score = calculate_metrics(np.array(predicted_labels), np.array(true_labels))
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='macro', zero_division=1.0)
    loss = log_loss(true_labels, predicted_labels)
    #classificaiton report 
    accuracy = accuracy_score(true_labels, predicted_labels)
    cm = confusion_matrix(true_labels, predicted_labels)
    print(f"Loss: {loss}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}, Acc: {accuracy}")
    print("confusion metrix:", cm)
    print("classification report:",classification_report(true_labels,predicted_labels, zero_division=1.0))

    return accuracy, precision, recall, f1


In [11]:

def predict_test(model, device):#, output_dir):
    candidate_list = ["positive", "neutral", "negative"]

    #model = MBartForConditionalGeneration.from_pretrained(output_dir)
    #model = MBartForConditionalGeneration.from_pretrained('/ICTeval/ICTeval_code/outputs/checkpoint-5865-epoch-15/mbart')
    model.eval()
    model.config.use_cache = False
    
    # tokenizer = MBart50TokenizerFast.from_pretrained('facebook/mbart-large-50')
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    #tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/mGPT")

    file_path = path
    with open(file_path, "r") as f:
        file = f.readlines()
    train_data = []
    count = 0
    total = 0
    true_labels = []
    predicted_labels = []
    for line in file:
        total += 1
        # score_list = []
        score_list1 = []
        score_list2 = []
        score_list3 = []
        score_list4 = []
        score_list5 = []
        line = line.strip()
        x, term, golden_polarity = line.split("\001")[0], line.split("\001")[1], line.split("\001")[2]
        input_ids = tokenizer([x] * 3, return_tensors='pt')['input_ids'].to(device)
        #sentence = tokenizer.decode([input_ids], skip_special_tokens=True)
        #print("sentence: ", sentence)
        #print("input_ID:", input_ids)
        # target_list = ["For " + term.lower() + ", the sentiment is " + candi.lower() + " ." for candi in candidate_list]
        # # target_list = ["For " + term.lower() + ", it is a " + candi.lower() + "sentence ." for candi in candidate_list]
        # # input_ids = tokenizer([x] * 3, return_tensors='pt')['input_ids']
        # output_ids = tokenizer(target_list, return_tensors='pt', padding=True, truncation=True)['input_ids']
        # with torch.no_grad():
        #     output = model(input_ids=input_ids.to(device), decoder_input_ids=output_ids.to(device))[0]
        #     logits = output.softmax(dim=-1).to('cpu').numpy()
        # for i in range(3):
        #     score = 1
        #     for j in range(logits[i].shape[0] - 2):
        #         score *= logits[i][j][output_ids[i][j + 1]]
        #     score_list1.append(score)

        target_list = ["The sentiment polarity of " + term.lower() + " is " + candi.lower() + " ." for candi in
                       candidate_list]
        #print("target_lsit",target_list)
        ## target_list = ["For " + term.lower() + ", it is a " + candi.lower() + "sentence ." for candi in candidate_list]
        ## input_ids = tokenizer([x] * 3, return_tensors='pt')['input_ids']

        output_ids = tokenizer(target_list, return_tensors='pt', padding=True, truncation=True)['input_ids'].to(device)
        model.to(device)
        output_ids = output_ids.to(device)
        print("logit and output_ID: ",output_ids)
        #sentence2 = tokenizer.decode(output_ids, skip_special_tokens=True)
        #print("output ID:", sentence2)
        #print("Model device:", model.device)
        #print("Input tensor device:", input_ids.device)
        #print("Output tensor device:", output_ids.device)
        with torch.no_grad():
            output = model(input_ids=input_ids, decoder_input_ids=output_ids)[0]
            #print(np.shape(output))
            logits = output.softmax(dim=-1).to('cpu').numpy()
            print("LOGIT:::",logits.shape)
        #with torch.no_grad():
        #    output = model(input_ids=input_ids.to(device))[0]
        #    logits = output.softmax(dim=-1).to('cpu').numpy()
        for i in range(3):
            score = 0
            for j in range(logits[i].shape[0] - 2):
                #print(logits[i][j][output_ids[i][j + 1]])
                score += np.log(logits[i][j][output_ids[i][j + 1]])
            score_list2.append(score)
        score_list = score_list2

        # target_list = ["The " + term.lower() + " category has a " + candi.lower() + " label ." for candi in
        #                candidate_list]
        # # target_list = ["For " + term.lower() + ", it is a " + candi.lower() + "sentence ." for candi in candidate_list]
        # # input_ids = tokenizer([x] * 3, return_tensors='pt')['input_ids']
        # output_ids = tokenizer(target_list, return_tensors='pt', padding=True, truncation=True)['input_ids']
        # with torch.no_grad():
        #     output = model(input_ids=input_ids.to(device), decoder_input_ids=output_ids.to(device))[0]
        #     logits = output.softmax(dim=-1).to('cpu').numpy()
        # for i in range(3):
        #     score = 1
        #     for j in range(logits[i].shape[0] - 2):
        #         score *= logits[i][j][output_ids[i][j + 1]]
        #     score_list3.append(score)

        # target_list = ["The sentiment is " + candi.lower() + " for " + term.lower() + " ." for candi in
        #                candidate_list]
        # # target_list = ["For " + term.lower() + ", it is a " + candi.lower() + "sentence ." for candi in candidate_list]
        # # input_ids = tokenizer([x] * 3, return_tensors='pt')['input_ids']
        # output_ids = tokenizer(target_list, return_tensors='pt', padding=True, truncation=True)['input_ids']
        # with torch.no_grad():
        #     output = model(input_ids=input_ids.to(device), decoder_input_ids=output_ids.to(device))[0]
        #     logits = output.softmax(dim=-1).to('cpu').numpy()
        # for i in range(3):
        #     score = 1
        #     for j in range(logits[i].shape[0] - 2):
        #         score *= logits[i][j][output_ids[i][j + 1]]
        #     score_list4.append(score)

        # target_list = ["The " + term.lower() + " is " + candi.lower() + " ." for candi in
        #                candidate_list]
        # # target_list = ["For " + term.lower() + ", it is a " + candi.lower() + "sentence ." for candi in candidate_list]
        # # input_ids = tokenizer([x] * 3, return_tensors='pt')['input_ids']
        # output_ids = tokenizer(target_list, return_tensors='pt', padding=True, truncation=True)['input_ids']
        # with torch.no_grad():
        #     output = model(input_ids=input_ids.to(device), decoder_input_ids=output_ids.to(device))[0]
        #     logits = output.softmax(dim=-1).to('cpu').numpy()
        # for i in range(3):
        #     score = 1
        #     for j in range(logits[i].shape[0] - 2):
        #         score *= logits[i][j][output_ids[i][j + 1]]
        #     score_list5.append(score)

        # score_list = [(score_list1[i] + score_list2[i]) for i in range(0, len(score_list1))]
        predict = candidate_list[np.argmax(score_list)]
        predicted_sentence = target_list[np.argmax(score_list)]
        true_labels.append(golden_polarity)
        predicted_labels.append(predict)
        # with open("ICTeval_predict_output/cnn_test2_og_backtrans_AI_predict.txt", 'a') as f0:
        #            f0.writelines(line + "\n")
        #            f0.writelines(predicted_sentence + "\n")
        # with open("ICTeval_predict_output/cnn_test2_og_backtrans_AI_true.txt", 'a') as f0:
        #            f0.writelines(line.split("\001")[0] + "\n")
        #            f0.writelines("The sentiment polarity of " + term.lower() + " is " + golden_polarity + " ."+ "\n")
        if predict == golden_polarity:
            count += 1
            print(line, term, " ", predicted_sentence,"actual:", predict, golden_polarity,"acc:", count/total, count, total)
        else:
            print("Mismatch:",line, " ", term , " ", predicted_sentence, " actual:", predict, golden_polarity,"acc: " ,count/total, count, total)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='macro', zero_division=1.0)    
    #precision, recall, f1_score = calculate_metrics(np.array(predicted_labels), np.array(true_labels))
    accuracy = accuracy_score(true_labels, predicted_labels)
    cm = confusion_matrix(true_labels, predicted_labels)
    print(f"Precision: {precision}, Recall: {recall}, F1-Score: {f1}, Acc: {accuracy}")
    print("confusion metrix:", cm)
    print("classification report:",classification_report(true_labels,predicted_labels, zero_division=1.0))
    #print(predicted_labels)
    #print(true_labels)

    result = {"File": file_path, "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1": f1}
    # return file_path, accuracy, precision, recall, f1
    return result


In [17]:
ls outputs/Original/1-fold

config.json             model.safetensors        tokenizer_config.json
generation_config.json  model_args.json          training_args.bin
merges.txt              special_tokens_map.json  vocab.json


In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model2 = BartForConditionalGeneration.from_pretrained(model_path).to(device)
# model2 = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn').to(device)

#MBartForConditionalGeneration
# model2 = GPT2LMHeadModel.from_pretrained("sberbank-ai/mGPT").to(device)
# model2 = GPT2Model.from_pretrained('outputs/mbart').to(device)
acc = predict_test(model2, device)

OSError: Incorrect path_or_model_id: '/outputs/Original/1-fold'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [16]:
acc

{'File': 'ICTeval_code/crossvalid/testformat/test4.txt',
 'Accuracy': 0.7788778877887789,
 'Precision': 0.9262926292629263,
 'Recall': 0.3333333333333333,
 'F1': 0.29189857761286336}

In [17]:
import os
print(os.getcwd())

/home/sinsin/ICTeval_code


In [18]:
!pwd

/home/sinsin/ICTeval_code
